<font color='blue'>Here I will document my method for cleaning my data files, which were taken from Kaggle's dataset “Comments on articles published in the New York Times” (https://www.kaggle.com/aashita/nyt-comments).

This code is broken into two sections - one cleans the data files with articles and one cleans the data files with comments. Here I test my cleaning using only a single file, but by concatenating data files together, I can run this code on the entire dataset.</font> 

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
sent_token = nltk.sent_tokenize
import csv  
from nltk import sent_tokenize, word_tokenize, pos_tag
import re
from sklearn.feature_extraction.text import CountVectorizer
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

#For all data - train = pd.read_csv("/root/Springboard/Data/cleaning/allArticles.csv")
train = pd.read_csv("/root/Springboard/Data/ArticlesApril2017.csv")

#train.head(5)

<font color='blue'>Initial goals: 

-Make sure the contents of each field are the correct type and have no missing data (i.e. scrub the 'NaN' from the 'abstract' field)

-Make sure that the data comes properly tokenized

-Convert all words to lowercase (to avoid confusion between uppercase and lowercase versions of the same word)

Several of these data columns (articleID, articleWordCount, multimedia, printPage) contain only integers or single lowercase words.</font> 

In [2]:
#for column in train:
    #print(train[column].get_dtype_counts())

<font color='blue'>From the above code, the only integer columns are 2, 7 and 9. The rest are string columns and need to be converted to lowercase. </font> 

In [24]:
train = pd.read_csv("/root/Springboard/Data/ArticlesApril2017.csv")
#train.head(5)

def clean_articles(doc):
    for index, column in enumerate(doc):
        if index == 2:
            doc[column] = doc[column].astype(str)
            continue
        if index == 7:
            doc[column] = doc[column].astype(str)
            continue
        if index == 9:
            doc[column] = doc[column].astype(str)
            continue
        doc[column] = doc[column].str.lower()
        #doc[column] = doc[column].str.strip()
        doc[column] = doc[column].replace(np.nan, '', regex=True)
        
    return doc

def norm_articles(doc):
    clean_articles(doc)
    doc3 = []
    for index, row in doc.iterrows():
        element = str(row.to_frame().T)
        #print(element)
        tokenData = nltk.word_tokenize(element)
        #print(tokenData)
        #print("EFFEW")
        filtered_tokens = [token for token in tokenData if token not in stop_words]
        doc2 = ' '.join(filtered_tokens)
        doc3.append(doc2)
    #print(type(doc3))
    return doc3

normalize_corpus = np.vectorize(norm_articles)

clean_art = clean_articles(train)
clean_art2 = norm_articles(train)

#print(clean_art.head(5))
print(clean_art2)

#print('\n'.join(clean))


["abstract articleID articleWordCount \\ 0 58def1347c459f24986d7c80 716 byline documentType \\ 0 stephen hiltner susan lehman article headline \\ 0 finding expansive view forgotten peop ... keywords multimedia newDesk \\ 0 [ 'photography ' , 'new york times ' , 'niger ' , 'fe ... 3 insider printPage pubDate sectionName \\ 0 2 2017-04-01 00:15:41 unknown snippet source \\ 0 one largest photo displays times ... new york times typeOfMaterial webURL 0 news https : //www.nytimes.com/2017/03/31/insider/nig ...", "abstract articleID articleWordCount byline \\ 1 58def3237c459f24986d7c84 823 gail collins documentType headline \\ 1 article , dreaded trump curse keywords multimedia newDesk \\ 1 [ 'united states politics government ' , 'tru ... 3 oped printPage pubDate sectionName \\ 1 23 2017-04-01 00:23:58 unknown snippet source typeOfMaterial \\ 1 meet gang bus . new york times op-ed webURL 1 https : //www.nytimes.com/2017/03/31/opinion/and ...", "abstract articleID articleWordCount byline \\ 2

<font color='blue'>Now we want a separate section to clean the comment files. </font> 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(clean_art2)
cv_matrix = cv_matrix.toarray()
cv_matrix

vocab = cv.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(clean_art2)

bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(clean_art2)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,876,877,878,879,880,881,882,883,884,885
0,1.000000,0.093023,0.075104,0.073160,0.102152,0.080041,0.064198,0.059141,0.071436,0.092527,...,0.021063,0.022543,0.021387,0.034122,0.037696,0.077088,0.028129,0.024588,0.025056,0.025521
1,0.093023,1.000000,0.121205,0.082356,0.114554,0.092221,0.070462,0.061832,0.079316,0.107438,...,0.078008,0.054097,0.052673,0.077746,0.056458,0.043082,0.104372,0.043228,0.043747,0.044559
2,0.075104,0.121205,1.000000,0.076434,0.092488,0.074456,0.056890,0.049921,0.064038,0.068681,...,0.017204,0.018601,0.038851,0.036888,0.031883,0.022053,0.055454,0.021697,0.022166,0.022577
3,0.073160,0.082356,0.076434,1.000000,0.090446,0.070862,0.065916,0.084581,0.072287,0.065605,...,0.081609,0.016920,0.016908,0.022987,0.023953,0.017724,0.015826,0.017230,0.016601,0.016909
4,0.102152,0.114554,0.092488,0.090446,1.000000,0.109739,0.078100,0.071595,0.100028,0.069382,...,0.024353,0.033460,0.024729,0.156533,0.043961,0.030823,0.026275,0.035338,0.035934,0.036601
5,0.080041,0.092221,0.074456,0.070862,0.109739,1.000000,0.060629,0.053202,0.068246,0.053621,...,0.018334,0.019823,0.018617,0.037459,0.033979,0.023503,0.019944,0.043526,0.126364,0.128710
6,0.064198,0.070462,0.056890,0.065916,0.078100,0.060629,1.000000,0.079186,0.084011,0.077625,...,0.020375,0.021870,0.020689,0.033286,0.042948,0.026131,0.021826,0.024548,0.024381,0.024834
7,0.059141,0.061832,0.049921,0.084581,0.071595,0.053202,0.079186,1.000000,0.078642,0.078812,...,0.027368,0.042776,0.021676,0.029470,0.037543,0.023263,0.020289,0.021505,0.020143,0.020517
8,0.071436,0.079316,0.064038,0.072287,0.100028,0.068246,0.084011,0.078642,1.000000,0.077091,...,0.020235,0.027370,0.020547,0.033057,0.035688,0.025951,0.021676,0.024379,0.024214,0.024663
9,0.092527,0.107438,0.068681,0.065605,0.069382,0.053621,0.077625,0.078812,0.077091,1.000000,...,0.020279,0.020606,0.020591,0.027995,0.023828,0.022171,0.096271,0.021674,0.020301,0.020678


In [ ]:
train = pd.read_csv("/root/Springboard/Data/CommentsApril2017.csv")

#for index, column in enumerate(train):
    #print(train[column].get_dtype_counts())
    #print(" " + str(index) + "\n")

In [17]:
#For all data - train = pd.read_csv("/root/Springboard/Data/cleaning/allComments.csv")
train = pd.read_csv("/root/Springboard/Data/CommentsApril2017.csv")

train.head(5)

strings = [1, 5, 10, 18, 22, 24, 25, 26, 29, 30, 33]

def clean_articles(doc):
    for index, column in enumerate(doc):
        if index in strings:
            doc[column] = doc[column].str.lower()
            #doc[column] = doc[column].str.strip()
            doc[column] = doc[column].replace(np.nan, '', regex=True)
        else:
            doc[column] = doc[column].astype(str)
            continue
       
        
    return doc

def norm_articles(doc):
    clean_articles(doc)
    doc3 = []
    for index, row in doc.iterrows():
        element = str(row.to_frame().T)
        #print(element)
        tokenData = nltk.word_tokenize(element)
        #print(tokenData)
        #print("EFFEW")
        filtered_tokens = [token for token in tokenData if token not in stop_words]
        doc2 = ' '.join(filtered_tokens)
        doc3.append(doc2)
    #print(type(doc3))
    return doc3

normalize_corpus = np.vectorize(norm_articles)

clean_comments = clean_articles(train)
#clean_comments2 = norm_articles(train)
#The second command takes awhile to run

clean_comments.head(5)

#print('\n'.join(clean))



,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,this project makes me happy to be a 30+ year t...,22022598.0,22022598,<br/>,comment,1491237056.0,1,False,0.0,...,"riverside, ca",,,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
1,1491188619,stunning photos and reportage. infuriating tha...,22017350.0,22017350,nan,comment,1491180489.0,1,False,0.0,...,<br/>,,,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
2,1491188617,brilliant work from conception to execution. i...,22017334.0,22017334,<br/>,comment,1491179470.0,1,False,0.0,...,raleigh nc,,,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
3,1491167820,nyt reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1491150196.0,1,False,0.0,...,"missouri, usa",,,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
4,1491167815,could only have been done in print. stunning.,22015466.0,22015466,<br/>,comment,1491147284.0,1,False,0.0,...,"tucson, arizona",,,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news


In [21]:
art_file_name = "cleaned_article_data.csv"
clean_art_csv = clean_art.to_csv(art_file_name, encoding='utf-8')

com_file_name = "cleaned_comment_data.csv"
clean_com_csv = clean_comments.to_csv(com_file_name, encoding='utf-8')